In [65]:
import pandas as pd

In [84]:
data = pd.read_excel('data_cleaning.xlsx', sheet_name = 'Usage', header = None, names = ['raw'])
df = data.copy()

In [85]:
df.head(10)

,raw
0,3 PM Mon 24th-Mar-2014___0.384 kwh
1,5AM 15-Aug-2014___1.201 kwh
2,__8PM Thu 20-Mar-2014____1.523 kwh
3,6PM 23rd-Apr-2014___0.424 kwh
4,_1AM Friday 19th-Dec-2014___0.209 kwh
5,_5AM Tue 19th-Aug-2014___1.228 kwh
6,12PM Mon 7th-Jul-2014___1.296 kwh
7,7 AM Tue 25-Nov-2014__0.437 kwh
8,_8AM 14-Aug-2014__0.523 kwh
9,__4PM 25th-Jan-2014__2.052kwh


<h1>Data Cleaning

In [86]:
df = pd.DataFrame(df.raw.replace('_', '', regex = True))                 # first, remove all underscores from data
df.head()

,raw
0,3 PM Mon 24th-Mar-20140.384 kwh
1,5AM 15-Aug-20141.201 kwh
2,8PM Thu 20-Mar-20141.523 kwh
3,6PM 23rd-Apr-20140.424 kwh
4,1AM Friday 19th-Dec-20140.209 kwh


In [87]:
df['time'] = df['raw'].str.extract(r'(.+[AP]M)')                             # get timestamp with regex    
df['time'] = df['time'].str.replace(' ', '')                                 # remove whitespaces
df.head()

,raw,time
0,3 PM Mon 24th-Mar-20140.384 kwh,3PM
1,5AM 15-Aug-20141.201 kwh,5AM
2,8PM Thu 20-Mar-20141.523 kwh,8PM
3,6PM 23rd-Apr-20140.424 kwh,6PM
4,1AM Friday 19th-Dec-20140.209 kwh,1AM


In [88]:
df['raw'] = df['raw'].str.replace('th', '')                                   # remove th, rd, st, nd to write simpler regex
df['raw'] = df['raw'].str.replace('rd', '')
df['raw'] = df['raw'].str.replace('st', '')
df['raw'] = df['raw'].str.replace('nd', '')

df['date'] = df['raw'].str.extract(r'([0-9]{1,2}-[A-Za-z]{3}-[0-9]{4})')      # regex for date
df['date'] = df['date'].str.strip()

df['day'] = df['date'].str.extract(r'([0-9]{1,2})')                           # regex for day
df['month'] = df['date'].str.extract(r'([A-Za-z]{3})')                        # regex for month

df['date'] = pd.to_datetime(df['date'])                                       # convert to datetime for day name
df['day_of_week'] = df['date'].dt.day_name()                                  # get dayname

df.head()

,raw,time,date,day,month,day_of_week
0,3 PM Mon 24-Mar-20140.384 kwh,3PM,2014-03-24,24,Mar,Monday
1,5AM 15-Aug-20141.201 kwh,5AM,2014-08-15,15,Aug,Friday
2,8PM Thu 20-Mar-20141.523 kwh,8PM,2014-03-20,20,Mar,Thursday
3,6PM 23-Apr-20140.424 kwh,6PM,2014-04-23,23,Apr,Wednesday
4,1AM Friday 19-Dec-20140.209 kwh,1AM,2014-12-19,19,Dec,Friday


In [89]:
df['usage'] = df['raw'].str.extract(r'([0-9]{1}\.[0-9]{2,3})')              # regex for electricity usage
df.head()

,raw,time,date,day,month,day_of_week,usage
0,3 PM Mon 24-Mar-20140.384 kwh,3PM,2014-03-24,24,Mar,Monday,0.384
1,5AM 15-Aug-20141.201 kwh,5AM,2014-08-15,15,Aug,Friday,1.201
2,8PM Thu 20-Mar-20141.523 kwh,8PM,2014-03-20,20,Mar,Thursday,1.523
3,6PM 23-Apr-20140.424 kwh,6PM,2014-04-23,23,Apr,Wednesday,0.424
4,1AM Friday 19-Dec-20140.209 kwh,1AM,2014-12-19,19,Dec,Friday,0.209


In [90]:
df.drop('raw', axis = 1, inplace = True)
df.head()

,time,date,day,month,day_of_week,usage
0,3PM,2014-03-24,24,Mar,Monday,0.384
1,5AM,2014-08-15,15,Aug,Friday,1.201
2,8PM,2014-03-20,20,Mar,Thursday,1.523
3,6PM,2014-04-23,23,Apr,Wednesday,0.424
4,1AM,2014-12-19,19,Dec,Friday,0.209


In [91]:
df.dtypes   # check data types                                                 

time                   object
date           datetime64[ns]
day                    object
month                  object
day_of_week            object
usage                  object
dtype: object

In [92]:
df['usage'] = df['usage'].astype(str).astype(float)        # convert usage to float

In [93]:
df.dtypes

time                   object
date           datetime64[ns]
day                    object
month                  object
day_of_week            object
usage                 float64
dtype: object

In [96]:
df.isnull().sum().sum()                                 # check NAs

0

In [97]:
df.loc[df.duplicated()]                                 # check duplicates

,time,date,day,month,day_of_week,usage
15,5AM,2014-08-15,15,Aug,Friday,1.201
16,8PM,2014-03-20,20,Mar,Thursday,1.523
17,6PM,2014-04-23,23,Apr,Wednesday,0.424
18,1AM,2014-12-19,19,Dec,Friday,0.209
19,5AM,2014-08-19,19,Aug,Tuesday,1.228
20,12PM,2014-07-07,7,Jul,Monday,1.296
